In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from build_model import interaction_model, get_subnetwork
from utils import get_interaction_model
from get_performance import get_data
from sklearn.model_selection import KFold, StratifiedKFold, ParameterSampler, train_test_split

In [4]:
from path_explain import utils, PathExplainerTF, scatter_plot, summary_plot
utils.set_up_environment(visible_devices='2')

In [15]:
x_train, y_train, x_test, y_test = get_data('pulsar')

In [17]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(x_train.shape[1],)))
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=1,  activation=None))
model.add(tf.keras.layers.Activation(tf.keras.activations.sigmoid))

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
# loss = tf.keras.losses.MeanSquaredError()
# metrics = [tf.keras.metrics.MeanSquaredError()]
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.AUC()]
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [19]:
model.fit(x_train, y_train, batch_size=300, epochs=10, verbose=1, validation_data=(x_test, y_test))

Train on 14318 samples, validate on 3580 samples
Epoch 1/10
14318/14318 [==============================] - 1s 73us/sample - loss: 0.1359 - auc_3: 0.9517 - val_loss: 0.0671 - val_auc_3: 0.9684
Epoch 2/10
14318/14318 [==============================] - 0s 17us/sample - loss: 0.0755 - auc_3: 0.9755 - val_loss: 0.0608 - val_auc_3: 0.9776
Epoch 3/10
14318/14318 [==============================] - 0s 18us/sample - loss: 0.0733 - auc_3: 0.9764 - val_loss: 0.0599 - val_auc_3: 0.9743
Epoch 4/10
14318/14318 [==============================] - 0s 19us/sample - loss: 0.0697 - auc_3: 0.9783 - val_loss: 0.0592 - val_auc_3: 0.9755
Epoch 5/10
14318/14318 [==============================] - 0s 18us/sample - loss: 0.0691 - auc_3: 0.9796 - val_loss: 0.0578 - val_auc_3: 0.9778
Epoch 6/10
14318/14318 [==============================] - 0s 18us/sample - loss: 0.0686 - auc_3: 0.9803 - val_loss: 0.0579 - val_auc_3: 0.9779
Epoch 7/10
14318/14318 [==============================] - 0s 18us/sample - loss: 0.0683 - auc

In [20]:
pair_model = interaction_model(num_features=x_train.shape[1],
                               num_layers=2,
                               hidden_layer_size=4,
                               num_outputs=1,
                               activation_function=tf.keras.activations.relu,
                               interactions_to_ignore=None,
                               regression=False)

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
# loss = tf.keras.losses.MeanSquaredError()
# metrics = [tf.keras.metrics.MeanSquaredError()]
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.AUC()]
pair_model.compile(optimizer=optimizer,
                   loss=loss,
                   metrics=metrics)

In [22]:
pair_model.fit(x_train, y_train, batch_size=300, epochs=10, verbose=1, validation_data=(x_test, y_test))

Train on 14318 samples, validate on 3580 samples
Epoch 1/10
14318/14318 [==============================] - 6s 430us/sample - loss: 0.2438 - auc_4: 0.8895 - val_loss: 0.0795 - val_auc_4: 0.9607
Epoch 2/10
14318/14318 [==============================] - 1s 87us/sample - loss: 0.0879 - auc_4: 0.9684 - val_loss: 0.0709 - val_auc_4: 0.9663
Epoch 3/10
14318/14318 [==============================] - 1s 94us/sample - loss: 0.0825 - auc_4: 0.9715 - val_loss: 0.0686 - val_auc_4: 0.9684
Epoch 4/10
14318/14318 [==============================] - 1s 93us/sample - loss: 0.0790 - auc_4: 0.9730 - val_loss: 0.0669 - val_auc_4: 0.9706
Epoch 5/10
14318/14318 [==============================] - 1s 96us/sample - loss: 0.0766 - auc_4: 0.9735 - val_loss: 0.0655 - val_auc_4: 0.9739
Epoch 6/10
14318/14318 [==============================] - 1s 91us/sample - loss: 0.0754 - auc_4: 0.9750 - val_loss: 0.0623 - val_auc_4: 0.9719
Epoch 7/10
14318/14318 [==============================] - 1s 90us/sample - loss: 0.0745 - au